In [1]:
%reload_ext autoreload
%autoreload 3

import matplotlib.pyplot as plt
from nilearn import datasets, image, plotting

import pandas as  pd
import matplotlib.pyplot as plt
import numpy as np
from nilearn import datasets, image, plotting
from nilearn.datasets import load_mni152_template
from nilearn.maskers.nifti_spheres_masker import _apply_mask_and_get_affinity
from nilearn.masking import intersect_masks
from src.acnets.parcellations.dosenbach import load_dosenbach2010_masker
from src.acnets.parcellations.friedman import load_fiedman2020_masker
from src.acnets.parcellations.gordon import \
    fetch_atlas as fetch_gordon2014_atlas
from src.acnets.parcellations import seitzman
from tqdm.auto import tqdm

tqdm.pandas()

In [2]:
# load MNI152 template
mni152 = load_mni152_template(resolution=2)
# nib.save(mni152, "mni152_2mm.nii.gz")

# load dosenbach2010
dosenbach2010_masker, dosenbach2010_labels = load_dosenbach2010_masker()
# dosenbach2010_labels['network_index'] = dosenbach2010_labels.groupby('network').ngroup()

from nilearn import image

# create mni152 boolean mask
mni152_data = mni152.get_fdata()[..., np.newaxis]
mni152_data = np.where(mni152_data > 0.0001, 1, 0).astype("int32")
mni152_mask = image.new_img_like(mni152, mni152_data)

In [3]:
gordon2014_atlas = fetch_gordon2014_atlas()


# # DEBUG
# plotting.plot_surf_roi(fsaverage['pial_left'], _surf, title='Gordon2014', colorbar=True)
# plt.show()


# gordon_labels = gordon2014_atlas.labels.reset_index()
# gordon_labels['network_index'] = gordon_labels['network'].astype('category').cat.codes + 1

# gordon_network_names = gordon_labels.network.astype('category').cat.categories.to_list()

# def get_network_index(region_index):
#     net_index = 0.0
#     if region_index > 0:
#         net_index = gordon_labels[['network_index']].values[int(region_index)-1,0]
#         net_index = net_index.astype('float')

#     return net_index

# gordon_network_maps = image.load_img(gordon2014_atlas.maps).get_fdata()
# gordon_network_maps = np.vectorize(get_network_index)(gordon_network_maps)
# gordon_network_maps = image.new_img_like(gordon2014_atlas.maps, gordon_network_maps)
# gordon_network_maps = image.resample_to_img(gordon_network_maps, mni152)

In [4]:


def create_network_img(regions):
    seeds = regions[['x','y','z']].values
    region_names = regions.index.to_list()
    X, A = _apply_mask_and_get_affinity(seeds, mni152_mask, radius=5, allow_overlap=False)
    A = A.toarray().sum(axis=0).reshape(mni152_mask.shape)

    # for i in np.arange(A.shape[0]):
    #     A[i,:] = np.where(A[i,:] > 0, i, 0)

    img = image.new_img_like(mni152, A)
    return img

dosenbach_network_images = dosenbach2010_labels.groupby('network').progress_apply(create_network_img)

  0%|          | 0/6 [00:00<?, ?it/s]

In [5]:
import nibabel as nib

networks_img = nib.concat_images(dosenbach_network_images.to_list())
nib.save(networks_img, 'dosenbach2010.nii.gz')
networks_img.shape

(99, 117, 95, 1, 6)

In [6]:

fsaverage = datasets.fetch_surf_fsaverage(mesh='fsaverage7')

suit = image.load_img('/home/morteza/workspace/acnets/atl-Anatom_dseg.label.gii')
suit

AttributeError: 'GiftiImage' object has no attribute '_data_cache'

In [ ]:
from nilearn.surface import vol_to_surf

# nki = datasets.fetch_surf_nki_enhanced()

surf_mesh = fsaverage.pial_left

mni152_surf = vol_to_surf(mni152, fsaverage['pial_left'], radius=1)
# dosenbach2010_surf = vol_to_surf(networks_img, surf_mesh, interpolation='nearest', radius=5)


In [ ]:

# # DEBUG
display = plotting.plot_surf_roi(suit, mni152_surf,
                       title=f'Dosenbach2010 - Sensorimotor',
                       bg_map=fsaverage.sulc_left, bg_on_data=False,
                       alpha=.5,
                       engine='matplotlib', 
                       colorbar=True,
                       # continuous colormap: viridis, plasma, inferno, magma, cividis
                       cmap='magma',
                       view='lateral')

# cbar.set_ticklabels(region_names)

plt.show()
# 'anterior', 'posterior', 'medial', 'lateral', 'dorsal', 'ventral'